In [1]:
#!fsharp
#!value --from-file input.txt --name input

In [1]:
#!fsharp
#!share input --from value

let parseRecords (records: string) = 
  records.Split("\n\n", System.StringSplitOptions.RemoveEmptyEntries)
  |> Seq.map (fun s -> 
    s.Split([|'\n'; ' '|], System.StringSplitOptions.RemoveEmptyEntries)
    |> Seq.map ((fun x -> x.Split(':')) >> function [|a ; b|] -> a,b | e -> failwith <| sprintf "%A" e)
    |> Map.ofSeq
  ) |> List.ofSeq

let passportRecords = parseRecords input
  

passportRecords

index,value
0,"[ [byr, 1926], [cid, 61], [ecl, hzl], [eyr, 2021], [hcl, #7d3b0c], [hgt, 186cm], [iyr, 2010], [pid, 221225902] ]"
1,"[ [byr, 1926], [cid, 92], [ecl, blu], [eyr, 2020], [hcl, #efcc98], [hgt, 178], [iyr, 2010], [pid, 433543520] ]"
2,"[ [byr, 1946], [ecl, brn], [eyr, 2026], [hcl, #b6652a], [hgt, 158cm], [iyr, 2018], [pid, 822320101] ]"
3,"[ [byr, 2008], [ecl, grn], [eyr, 1937], [hcl, z], [hgt, 138], [iyr, 2010], [pid, 21019503] ]"
4,"[ [byr, 2018], [cid, 75], [ecl, #d06796], [eyr, 1990], [hcl, z], [hgt, 176in], [iyr, 2019], [pid, 153cm] ]"
5,"[ [byr, 1994], [cid, 80], [ecl, grn], [eyr, 2024], [hcl, #ceb3a1], [hgt, 176cm], [iyr, 2020], [pid, 665071929] ]"
6,"[ [byr, 1955], [cid, 280], [ecl, blu], [eyr, 2013], [hcl, #733820], [hgt, 155cm], [iyr, 2011], [pid, 2346820632] ]"
7,"[ [byr, 2015], [ecl, brn], [eyr, 2026], [hcl, #4a5917], [hgt, 61cm], [iyr, 2026], [pid, 4772651050] ]"
8,"[ [byr, 1974], [ecl, gry], [eyr, 2024], [hcl, #a97842], [hgt, 182cm], [iyr, 2019], [pid, 917294399] ]"
9,"[ [byr, 2026], [cid, 141], [ecl, #9c635c], [eyr, 1998], [hcl, z], [hgt, 175cm], [iyr, 2010], [pid, 830491851] ]"


In [1]:
#!fsharp
let hasRequiredKeys (passportRecord: Map<string, string>) = 
  let requiredKeys = [
    "byr";
    "iyr";
    "eyr";
    "hgt";
    "hcl";
    "ecl";
    "pid";
    // "cid";
  ]
  requiredKeys |> List.forall passportRecord.ContainsKey

passportRecords
|> List.countBy hasRequiredKeys

index,Item1,Item2
0,True,237
1,False,53


Rules about what values are valid for automatic validation:

* byr (Birth Year) - four digits; at least 1920 and at most 2002.
* iyr (Issue Year) - four digits; at least 2010 and at most 2020.
* eyr (Expiration Year) - four digits; at least 2020 and at most 2030.
* hgt (Height) - a number followed by either cm or in:
* If cm, the number must be at least 150 and at most 193.
* If in, the number must be at least 59 and at most 76.
* hcl (Hair Color) - a # followed by exactly six characters 0-9 or a-f.
* ecl (Eye Color) - exactly one of: amb blu brn gry grn hzl oth.
* pid (Passport ID) - a nine-digit number, including leading zeroes.
* cid (Country ID) - ignored, missing or not.

In [1]:
#!fsharp
open System.Text.RegularExpressions

let (|Int|_|) s = 
    try 
        Some (int s)
    with _ -> None

let (|InRange|_|) min max =
  function
  | i when i >= min && i <= max -> Some i
  | _ -> None

let (|Match|_|) pattern s =
  let m = Regex.Match(s, pattern)
  if m.Success then Some(List.tail [ for g in m.Groups -> g.Value ])
  else None

type Height = Cm of int | Inches of int
let (|Measure|_|) =
  function
  | "cm" -> Some Cm
  | "in" -> Some Inches
  | _ -> None

let (|Height|_|) = 
  function
  | Match @"(\d+)(cm|in)" [Int i; Measure measure] -> Some (measure i)
  | _ -> None

let validate (passport: Map<string, string>) = 
  let errors = 
    [
      ("byr", function Some (Int (InRange 1920 2002 _)) -> None | e -> Some e)
      ("iyr", function Some (Int (InRange 2010 2020 _)) -> None | e -> Some e)
      ("eyr", function Some (Int (InRange 2020 2030 _)) -> None | e -> Some e)
      ("hgt", function Some (Height (Cm (InRange 150 193 _)) | Height (Inches (InRange 59 76 _))) -> None | e -> Some e)
      ("hcl", function Some (Match "#[0-9a-f]{6}" _) -> None | e -> Some e)
      ("ecl", function Some ("amb" | "blu" | "brn" | "gry" | "grn" | "hzl" | "oth") -> None | e -> Some e)
      ("pid", function Some (Match @"^\d{9}$" _) -> None | e -> Some e)
    ] 
    |> List.choose (fun (key, validator) -> 
      passport 
      |> Map.tryFind key 
      |> validator 
      |> Option.map (function Some badValue -> (key, badValue) | None -> (key, "<null>")))
  if (List.isEmpty errors) then Ok passport else Error errors

let isOk = function Ok _ -> true | Error _ -> false  


In [1]:
#!fsharp
let validations = passportRecords |> List.map validate

validations |> List.countBy isOk

index,Item1,Item2
0,True,172
1,False,118
